In [10]:
import ctypes
import logging
import multiprocessing as mp

from contextlib import closing
import numpy as np
from datetime import datetime
import os
import cv2
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions

In [6]:
info = mp.get_logger().info

In [11]:
from scipy.misc import imread, imsave
from sklearn.externals.joblib import Parallel, delayed
from PIL import Image
from skimage import img_as_float
import multiprocess as mp
import cv2

def process_images(image_paths_list):
    
    model = VGG16(weights='imagenet', include_top=False)
    
    images = Parallel(n_jobs=6, verbose=5)(
        delayed(Image.open)(f) for f in image_paths_list
    )   

    new_images = []
    def resize_process(task_info):
        im = task_info
        im_resized = im.resize((224, 224), Image.ANTIALIAS) 
        im_arr = np.expand_dims(im_resized, axis=0) # add one dimension as 1
        im_arr.flags.writeable = True
        im_arr = im_arr.astype(np.float64)
        return im_arr
    
    def resize_organizer():
        task_list = []
        for im in images:
            task_list.append(im)
        p = mp.Pool(mp.cpu_count())
        new_images = p.map(resize_process, task_list)
        return new_images

#     images = Parallel(n_jobs=6, verbose=5)(
#         delayed(img_as_float)(f) for f in images
#     )
    
#     task_list2 = []
#     features = []
#     def predict_process(task_info):
#         im = task_info
#         features.append(model.predict(im))
        
#     p = mp.Pool(mp.cpu_count())
#     features = p.map(predict_process(), images)
    new_images = resize_organizer()
    return new_images

In [6]:
image_paths_list = []
for i in range(1000):
    path = os.getcwd() + '/datasets/frames/video' + str(i)
    if not os.path.exists(path): continue
    image_paths_list.extend([path + \
                             '/frame' +str(idx)+'.jpg' for idx in range(1, 11, 1)])
image_paths_list[:10]

['/home/sunjiajun/cs231n_project/datasets/frames/video0/frame1.jpg',
 '/home/sunjiajun/cs231n_project/datasets/frames/video0/frame2.jpg',
 '/home/sunjiajun/cs231n_project/datasets/frames/video0/frame3.jpg',
 '/home/sunjiajun/cs231n_project/datasets/frames/video0/frame4.jpg',
 '/home/sunjiajun/cs231n_project/datasets/frames/video0/frame5.jpg',
 '/home/sunjiajun/cs231n_project/datasets/frames/video0/frame6.jpg',
 '/home/sunjiajun/cs231n_project/datasets/frames/video0/frame7.jpg',
 '/home/sunjiajun/cs231n_project/datasets/frames/video0/frame8.jpg',
 '/home/sunjiajun/cs231n_project/datasets/frames/video0/frame9.jpg',
 '/home/sunjiajun/cs231n_project/datasets/frames/video0/frame10.jpg']

In [12]:
im = process_images(image_paths_list[:800])

ImportError: [joblib] Attempting to do parallel computing without protecting your import on a system that does not support forking. To use parallel-computing in a script, you must protect your main loop using "if __name__ == '__main__'". Please see the joblib documentation on Parallel for more information